# Fast.ai Machine Learning - Course 1, Lesson 1

## 0 - Imports

In [ ]:
import pandas as pd
import numpy as np
import os
from pandas.api.types import is_string_dtype, is_object_dtype

In [ ]:
# change current working directory
os.chdir('..')
print(f'cwd: {os.getcwd()}')

## 1 - Downloading the Data

This section only needs to be run once.

Downloading data from Kaggle requires us to:
1. Install Kaggle
2. Create a Kaggle folder in our home directory (it'll be hidden)
3. Get our API credentials from the Kaggle 'Settings' page
4. Place the credentials (.json file) in the Kaggle folder from step 2

In [ ]:
!pip install -q kaggle

In [ ]:
# create a kaggle directory
dir = os.path.expanduser('~/.kaggle')
os.makedirs(dir, exist_ok=True)

In [ ]:
# copy credentials to kaggle folder
creds = '/Users/chelseatucker/credentials/kaggle.json'
!cp $creds ~/.kaggle

# change permissions so only I have read & write access to the credentials file
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# create a bulldozers directory
os.makedirs('data/bbfb', exist_ok=True)

# downloading the bulldozers dataset to the 'data' folder
!kaggle competitions download -c bluebook-for-bulldozers -p 'data/bbfb'

In [ ]:
# # unzip the data
!unzip -q data/bbfb/bluebook-for-bulldozers.zip -d 'data/bbfb'

# unzip train data
!unzip -q data/bbfb/Train.zip -d 'data/bbfb'

## 2 - Exploring the Data

In [ ]:
df_raw = pd.read_csv('data/bbfb/Train.csv', 
                     low_memory=False, 
                     parse_dates=['saledate'])

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# quick look to see if the data has imported correctly
df_raw.head(3)

In [ ]:
from src.eda import df_look

# high level overview of the data
df_look(df_raw)

## 3 - Feature Engineering

### 3.1 - Data Edits

The competiton wants us to use RMSLE as the measure between actuals and predictions so we'll take the log of the dependent variable.

In [ ]:
df_raw.SalePrice = np.log(df_raw.SalePrice)

Individual edits:

In [ ]:
# looking at individual levels
df_raw.Blade_Width.value_counts(dropna=False)

In [ ]:
# convert column to string
df_raw.Blade_Width = df_raw.Blade_Width.astype('string', inplace=True)

### 3.x - Feature Creation

Extracting more information from date/time columns:

In [ ]:
from src.preprocessing import add_dateattr

# extracting more information from the date field
add_dateattr(df_raw, 'saledate')

### 3.x - Categorical Features

In [ ]:
# list object/string columns
cats = []
for c in df_raw.columns:
    if is_string_dtype(df_raw[c]) or is_object_dtype(df_raw[c]): 
        cats.append(c)

cats

In [ ]:
from src.preprocessing import conv_to_cat

# converting all string/object columns to categories
conv_to_cat(df_raw)

In [ ]:
# checking category orders
for c in cats:
    print(c,':',df_raw[c].cat.categories)
    print()

In [ ]:
# reordering categories
df_raw.UsageBand = df_raw.UsageBand.cat.reorder_categories(['Low', 'Medium', 'High'], ordered=True)
df_raw.ProductSize = df_raw.ProductSize.cat.reorder_categories(['Mini', 'Small', 'Compact', 'Medium', 'Large / Medium', 'Large'], ordered=True)
df_raw.Blade_Width = df_raw.Blade_Width.cat.reorder_categories(["<12'", "12'", "13'", "14'", "16'", "None or Unspecified"], ordered=True)
df_raw.Tire_Size = df_raw.Tire_Size.reorder_categories(['Low', 'Medium', 'High'], ordered=True)

# checking after reordering
df_raw.UsageBand.cat.categories